![read_agent_teaser](https://read-agent.github.io/img/teaser.png)

From https://github.com/read-agent/read-agent.github.io/blob/main/assets/read_agent_demo.ipynb

In [1]:
# !wget https://github.com/nyu-mll/quality/raw/main/data/v1.0.1/QuALITY.v1.0.1.htmlstripped.dev
import re, time, datetime, json, string, copy

In [2]:
# # @title Using OpenAI GPT model (DO NOT run the next cell if using GPT)
# # !pip3 install openai
# import openai

# # key = 'YOUR API KEY'  #@param {type: "string"}
# import os
# import dotenv
# dotenv.load_dotenv()
# key = os.getenv('OPENAI_API_KEY')
# gpt_client = openai.OpenAI(api_key=key)
# model_type = 'gpt'

# def query_gpt_model(
#     prompt: str,
#     lm: str = 'gpt-3.5-turbo-1106',
#     temperature: float = 0.0,
#     max_decode_steps: int = 512,
#     seconds_to_reset_tokens: float = 30.0,
# ) -> str:
#   while True:
#     try:
#       raw_response = gpt_client.chat.completions.with_raw_response.create(
#         model=lm,
        
#         max_tokens=max_decode_steps,
#         temperature=temperature,
#         messages=[
#           {'role': 'user', 'content': prompt},
#         ]
#       )
#       completion = raw_response.parse()
#       return completion.choices[0].message.content
#     except openai.RateLimitError as e:
#       print(f'{datetime.datetime.now()}: query_gpt_model: RateLimitError {e.message}: {e}')
#       time.sleep(seconds_to_reset_tokens)
#     except openai.APIError as e:
#       print(f'{datetime.datetime.now()}: query_gpt_model: APIError {e.message}: {e}')
#       print(f'{datetime.datetime.now()}: query_gpt_model: Retrying after 5 seconds...')
#       time.s
# leep(5)
from genai.llms.simple_query import ModelManager

model_manager = ModelManager()


model_type = 'gpt-3.5-turbo-1106'
llm = model_manager.get_llm(model_type )
llm_query = llm.query

In [3]:
# # @title Using Google Gemini model (DO NOT run this if using GPT)
# !pip3 install -q -U google-generativeai
# import google.generativeai as genai

# key = 'YOUR API KEY'  #@param {type: "string"}

# genai.configure(api_key=key)
# model = genai.GenerativeModel('gemini-pro')
# model_type = 'gemini'

# def query_gemini_model(
#     prompt: str,
#     retries: int = 10,
# ) -> str:
#   while True and retries > 0:
#     try:
#       response = model.generate_content(prompt)
#       text_response = response.text.replace("**", "")
#       return text_response
#     except Exception as e:
#       print(f'{datetime.datetime.now()}: query_gemini_model: Error: {e}')
#       print(f'{datetime.datetime.now()}: query_gemini_model: Retrying after 5 seconds...')
#       retries -= 1
#       time.sleep(5)
# from genai.llms.simple_query import query_gemini_model

In [4]:
# def query_model(prompt):
#   if model_type == "gpt":
#     return query_gpt_model(prompt)
#   elif model_type == "gemini":
#     return query_gemini_model(prompt)


In [5]:
#@title Load a QuALITY example

# Fields that are straight text copies from raw example to processed example.
_ONE2ONE_FIELDS = (
    'article',
    'article_id',
    'set_unique_id',
    'writer_id',
    'source',
    'title',
    'topic',
    'url',
    'writer_id',
    'author',
)

quality_dev = []

with open('QuALITY.v1.0.1.htmlstripped.dev', 'r') as f:
  for line in f.readlines():
    j = json.loads(line)
    fields = {k: j[k] for k in _ONE2ONE_FIELDS}
    fields.update({
        'questions': [q['question'] for q in j['questions']],
        'question_ids': [q['question_unique_id'] for q in j['questions']],
        'difficults': [q['difficult'] for q in j['questions']],
        'options': [q['options'] for q in j['questions']],
    })

    fields.update({
        'gold_labels': [q['gold_label'] for q in j['questions']],
        'writer_labels': [q['writer_label'] for q in j['questions']],
      })

    quality_dev.append(fields)

example = quality_dev[13]

In [6]:
# #@title Helper functions

# all_lowercase_letters = string.ascii_lowercase  # "abcd...xyz"
# bracketed_lowercase_letters_set = set(
#     [f"({l})" for l in all_lowercase_letters]
# )  # {"(a)", ...}
# bracketed_uppercase_letters_set = set(
#     [f"({l.upper()})" for l in all_lowercase_letters]
# )  # {"(a)", ...}

# choices = ['(A)', '(B)', '(C)', '(D)']

# def get_index_from_symbol(answer):
#   """Get the index from the letter symbols A, B, C, D, to extract answer texts.

#   Args:
#     answer (str): the string of answer like "(B)".

#   Returns:
#     index (int): how far the given choice is from "a", like 1 for answer "(B)".
#   """
#   answer = str(answer).lower()
#   # extract the choice letter from within bracket
#   if answer in bracketed_lowercase_letters_set:
#     answer = re.findall(r"\(.*?\)", answer)[0][1]
#   index = ord(answer) - ord("a")
#   return index

# def count_words(text):
#   """Simple word counting."""
#   return len(text.split())

# def quality_gutenberg_parser(raw_article):
#   """Parse Gutenberg articles in the QuALITY dataset."""
#   lines = []
#   previous_line = None
#   for i, line in enumerate(raw_article.split('\n')):
#     line = line.strip()
#     original_line = line
#     if line == '':
#       if previous_line == '':
#         line = '\n'
#       else:
#         previous_line = original_line
#         continue
#     previous_line = original_line
#     lines.append(line)
#   return ' '.join(lines)

In [30]:
%load_ext autoreload
%autoreload 2
from genai.text_processes.pagination import quality_pagination
# #@title ReadAgent (1) Episode Pagination

# prompt_pagination_template = """
# You are given a passage that is taken from a larger text (article, book, ...) and some numbered labels between the paragraphs in the passage.
# Numbered label are in angeled brackets. For example, if the label number is 19, it shows as <19> in text.
# Please choose one label that it is natural to break reading.
# Such point can be scene transition, end of a dialogue, end of an argument, narrative transition, etc.
# Please answer the break point label and explain.
# For example, if <57> is a good point to break, answer with \"Break point: <57>\n Because ...\"

# Passage:

# {0}
# {1}
# {2}

# """

# def parse_pause_point(text):
#   text = text.strip("Break point: ")
#   if text[0] != '<':
#     return None
#   for i, c in enumerate(text):
#     if c == '>':
#       if text[1:i].isnumeric():
#         return int(text[1:i])
#       else:
#         return None
#   return None


# def quality_pagination(example,
#                        word_limit=600,
#                        start_threshold=280,
#                        max_retires=10,
#                        verbose=True,
#                        allow_fallback_to_last=True):
#   article = example['article']
#   title = example['title']
#   print(f"[Pagination][Article {title}]")
#   paragraphs = quality_gutenberg_parser(article).split('\n')

#   i = 0
#   pages = []
#   while i < len(paragraphs):
#     preceding = "" if i == 0 else "...\n" + '\n'.join(pages[-1])
#     passage = [paragraphs[i]]
#     wcount = count_words(paragraphs[i])
#     j = i + 1
#     while wcount < word_limit and j < len(paragraphs):
#       wcount += count_words(paragraphs[j])
#       if wcount >= start_threshold:
#         passage.append(f"<{j}>")
#       passage.append(paragraphs[j])
#       j += 1
#     passage.append(f"<{j}>")
#     end_tag = "" if j == len(paragraphs) else paragraphs[j] + "\n..."

#     pause_point = None
#     if wcount < 350:
#       pause_point = len(paragraphs)
#     else:
#       prompt = prompt_pagination_template.format(preceding, '\n'.join(passage), end_tag)
#       response = query_model(prompt=prompt).strip()
#       pause_point = parse_pause_point(response)
#       if pause_point and (pause_point <= i or pause_point > j):
#         print(f"prompt:\n{prompt},\nresponse:\n{response}\n")
#         print(f"i:{i} j:{j} pause_point:{pause_point}")
#         pause_point = None
#       if pause_point is None:
#         if allow_fallback_to_last:
#           pause_point = j
#         else:
#           raise ValueError(f"prompt:\n{prompt},\nresponse:\n{response}\n")

#     page = paragraphs[i:pause_point]
#     pages.append(page)
#     if verbose:
#       print(f"Paragraph {i}-{pause_point-1}", page)
#     i = pause_point
#   print(f"[Pagination] Done with {len(pages)} pages")
#   return pages

# pages = quality_pagination(example, query_model=llm_query)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# #@title ReadAgent (2) Memory Gisting

# prompt_shorten_template = """
# Please shorten the following passage.
# Just give me a shortened version. DO NOT explain your reason.

# Passage:
# {}

# """

# def quality_gisting(example, pages, word_limit=600, start_threshold=280, verbose=True):
#   article = example['article']
#   title = example['title']
#   word_count = count_words(article)
#   print(f"[Gisting][Article {title}], {word_count} words")

#   shortened_pages = []
#   for i, page in enumerate(pages):
#     prompt = prompt_shorten_template.format('\n'.join(page))
#     response = query_model(prompt)
#     shortened_text = response.strip()
#     shortened_pages.append(shortened_text)
#     if verbose:
#       print("[gist] page {}:".format(i), shortened_text, flush=True)
#   shortened_article = '\n'.join(shortened_pages)
#   gist_word_count = count_words(shortened_article)
#   if verbose:
#     print("Shortened article:\n", shortened_article, flush=True)
#   output = copy.deepcopy(example)
#   output.update({'title': title, 'word_count': word_count, 'gist_word_count': gist_word_count, 'shortened_pages': shortened_pages, 'pages': pages})
#   if verbose:
#     print(f"compression rate {round(100.0 - gist_word_count/word_count*100, 2)}% ({gist_word_count}/{word_count})")
#   return output
from genai.text_processes.gisting import quality_gisting
example_with_gists = quality_gisting(example, pages, query_model=llm_query)

[Gisting][Article Off Course], 2712 words
[gist] page 0: The first envoy from another world was about to speak, but he seemed more interested in a horse. Patrolmen Dermott and Casey approached the strange craft and encountered Dameri Tass, who spoke in a language they didn't understand.
[gist] page 1: Dameri Tass emerged from his spacecraft with a strange contraption and tried to get the patrolmen to put on a metal cap. Despite their reluctance, they were instructed to humor him until officials arrived.
[gist] page 2: Casey put on the cap and felt a shock, thinking he had been murdered. His friend explained that the cap was harmless and allowed him to understand their language. The patrolman made a comment about their Irish accents.
[gist] page 3: Dameri Tass encounters a horse and then is taken to Washington, causing a major uproar.
[gist] page 4: The United Nations demanded the alien be heard, and the White House yielded. The world was excited and eagerly awaited the visitor's messag

In [13]:
from genai.text_processes.utils import count_words 


In [17]:
#@title ReadAgent (3) Look-Up

prompt_lookup_template = """
The following text is what you remembered from reading an article and a multiple choice question related to it.
You may read 1 to 6 page(s) of the article again to refresh your memory to prepare yourselve for the question.
Please respond with which page(s) you would like to read.
For example, if your only need to read Page 8, respond with \"I want to look up Page [8] to ...\";
if your would like to read Page 7 and 12, respond with \"I want to look up Page [7, 12] to ...\";
if your would like to read Page 2, 3, 7, 15 and 18, respond with \"I want to look up Page [2, 3, 7, 15, 18] to ...\".
if your would like to read Page 3, 4, 5, 12, 13 and 16, respond with \"I want to look up Page [3, 3, 4, 12, 13, 16] to ...\".
DO NOT select more pages if you don't need to.
DO NOT answer the question yet.

Text:
{}

Question:
{}
{}

Take a deep breath and tell me: Which page(s) would you like to read again?
"""

prompt_answer_template = """
Read the following article and answer a multiple choice question.
For example, if (C) is correct, answer with \"Answer: (C) ...\"

Article:
{}

Question:
{}
{}

"""

def quality_parallel_lookup(example, verbose=True):
  preprocessed_pages = example['pages']
  article = example['article']
  title = example['title']
  word_count = example['word_count']
  gist_word_count = example['gist_word_count']
  pages = example['pages']
  shortened_pages = example['shortened_pages']
  questions = example['questions']
  options = example['options']
  gold_labels = example['gold_labels']  # numerical [1, 2, 3, 4]

  print(f"[Look-Up][Article {title}] {word_count} words")

  model_choices = []
  lookup_page_ids = []

  shortened_pages_pidx = []
  for i, shortened_text in enumerate(shortened_pages):
    shortened_pages_pidx.append("<Page {}>\n".format(i) + shortened_text)
  shortened_article = '\n'.join(shortened_pages_pidx)

  expanded_gist_word_counts = []
  for i, label in enumerate(gold_labels):
    # only test the first question for demo
    if i != 1:
      continue
    q = questions[i]
    print("question: ", q)
    options_i = [f"{ol} {o}" for ol, o in zip(choices, options[i])]
    print("options: ", "\n".join(options_i))
    prompt_lookup = prompt_lookup_template.format(shortened_article, q, '\n'.join(options_i))

    page_ids = []

    response = query_model(prompt=prompt_lookup).strip()

    try: start = response.index('[')
    except ValueError: start = len(response)
    try: end = response.index(']')
    except ValueError: end = 0
    if start < end:
      page_ids_str = response[start+1:end].split(',')
      page_ids = []
      for p in page_ids_str:
        if p.strip().isnumeric():
          page_id = int(p)
          if page_id < 0 or page_id >= len(pages):
            print("Skip invalid page number: ", page_id, flush=True)
          else:
            page_ids.append(page_id)

    if verbose:
      print("Model chose to look up page {}".format(page_ids))

    # Memory expansion after look-up, replacing the target shortened page with the original page
    expanded_shortened_pages = shortened_pages[:]
    if len(page_ids) > 0:
      for page_id in page_ids:
        expanded_shortened_pages[page_id] = '\n'.join(pages[page_id])

    expanded_shortened_article = '\n'.join(expanded_shortened_pages)
    expanded_gist_word_count = count_words(expanded_shortened_article)
    if verbose:
      print("Expanded shortened article:\n", expanded_shortened_article, flush=True)
    prompt_answer = prompt_answer_template.format(expanded_shortened_article, q, '\n'.join(options_i))

    model_choice = None
    response = llm_query(prompt=prompt_answer)
    response = response.strip()
    for j, choice in enumerate(choices):
      if response.startswith(f"Answer: {choice}") or response.startswith(f"Answer: {choice[1]}"):
        model_choice = j+1
        break
    is_correct = 1 if model_choice == label else 0
    print(f"question: {q}")
    print(f"reference answer: {choices[label]}, model prediction: {choices[model_choice]}, is_correct: {is_correct}")
    print(f"compression rate {round(100.0 - gist_word_count/word_count*100, 2)}% ({gist_word_count}/{word_count})")
    print(f"compression rate after look-up {round(100.0 - expanded_gist_word_count/word_count*100, 2)}% ({expanded_gist_word_count}/{word_count})")

quality_parallel_lookup(example_with_gists)

[Look-Up][Article Off Course] 2712 words
question:  What happened to Dameri while he was in custody of the government?


NameError: name 'choices' is not defined

In [16]:
# #@title ReadAgent (3) Look-Up

# prompt_lookup_template = """
# The following text is what you remembered from reading an article and a multiple choice question related to it.
# You may read 1 to 6 page(s) of the article again to refresh your memory to prepare yourselve for the question.
# Please respond with which page(s) you would like to read.
# For example, if your only need to read Page 8, respond with \"I want to look up Page [8] to ...\";
# if your would like to read Page 7 and 12, respond with \"I want to look up Page [7, 12] to ...\";
# if your would like to read Page 2, 3, 7, 15 and 18, respond with \"I want to look up Page [2, 3, 7, 15, 18] to ...\".
# if your would like to read Page 3, 4, 5, 12, 13 and 16, respond with \"I want to look up Page [3, 3, 4, 12, 13, 16] to ...\".
# DO NOT select more pages if you don't need to.
# DO NOT answer the question yet.

# Text:
# {}

# Question:
# {}
# {}

# Take a deep breath and tell me: Which page(s) would you like to read again?
# """

# prompt_answer_template = """
# Read the following article and answer a multiple choice question.
# For example, if (C) is correct, answer with \"Answer: (C) ...\"

# Article:
# {}

# Question:
# {}
# {}

# """

# def quality_parallel_lookup(example, verbose=True):
#   preprocessed_pages = example['pages']
#   article = example['article']
#   title = example['title']
#   word_count = example['word_count']
#   gist_word_count = example['gist_word_count']
#   pages = example['pages']
#   shortened_pages = example['shortened_pages']
#   questions = example['questions']
#   options = example['options']
#   gold_labels = example['gold_labels']  # numerical [1, 2, 3, 4]

#   print(f"[Look-Up][Article {title}] {word_count} words")

#   model_choices = []
#   lookup_page_ids = []

#   shortened_pages_pidx = []
#   for i, shortened_text in enumerate(shortened_pages):
#     shortened_pages_pidx.append("<Page {}>\n".format(i) + shortened_text)
#   shortened_article = '\n'.join(shortened_pages_pidx)

#   expanded_gist_word_counts = []
#   for i, label in enumerate(gold_labels):
#     # only test the first question for demo
#     if i != 1:
#       continue
#     q = questions[i]
#     print("question: ", q)
#     options_i = [f"{ol} {o}" for ol, o in zip(model_choices, options[i])]
#     print("options: ", "\n".join(options_i))
#     prompt_lookup = prompt_lookup_template.format(shortened_article, q, '\n'.join(options_i))

#     page_ids = []

#     response = (prompt=prompt_lookup).strip()


#     try: start = response.index('[')
#     except ValueError: start = len(response)
#     try: end = response.index(']')
#     except ValueError: end = 0
#     if start < end:
#       page_ids_str = response[start+1:end].split(',')
#       page_ids = []
#       for p in page_ids_str:
#         if p.strip().isnumeric():
#           page_id = int(p)
#           if page_id < 0 or page_id >= len(pages):
#             print("Skip invalid page number: ", page_id, flush=True)
#           else:
#             page_ids.append(page_id)

#     if verbose:
#       print("Model chose to look up page {}".format(page_ids))

#     # Memory expansion after look-up, replacing the target shortened page with the original page
#     expanded_shortened_pages = shortened_pages[:]
#     if len(page_ids) > 0:
#       for page_id in page_ids:
#         expanded_shortened_pages[page_id] = '\n'.join(pages[page_id])

#     expanded_shortened_article = '\n'.join(expanded_shortened_pages)
#     expanded_gist_word_count = count_words(expanded_shortened_article)
#     if verbose:
#       print("Expanded shortened article:\n", expanded_shortened_article, flush=True)
#     prompt_answer = prompt_answer_template.format(expanded_shortened_article, q, '\n'.join(options_i))

#     model_choice = None
#     response = llm_query(prompt=prompt_answer)
#     response = response.strip()
#     for j, choice in enumerate(model_choices):
#       if response.startswith(f"Answer: {choice}") or response.startswith(f"Answer: {choice[1]}"):
#         model_choice = j+1
#         break
#     is_correct = 1 if model_choice == label else 0
#     print(f"question: {q}")
#     print(f"reference answer: {model_choices[label]}, model prediction: {model_choices[model_choice]}, is_correct: {is_correct}")
#     print(f"compression rate {round(100.0 - gist_word_count/word_count*100, 2)}% ({gist_word_count}/{word_count})")
#     print(f"compression rate after look-up {round(100.0 - expanded_gist_word_count/word_count*100, 2)}% ({expanded_gist_word_count}/{word_count})")

# quality_parallel_lookup(example_with_gists)

[Look-Up][Article Off Course] 2712 words
question:  What happened to Dameri while he was in custody of the government?
options:  
Model chose to look up page [6, 7, 8]
Expanded shortened article:
 The first envoy from another world was about to speak, but he seemed more interested in a horse. Patrolmen Dermott and Casey approached the strange craft and encountered Dameri Tass, who spoke in a language they didn't understand.
Dameri Tass emerged from his spacecraft with a strange contraption and tried to get the patrolmen to put on a metal cap. Despite their reluctance, they were instructed to humor him until officials arrived.
Casey put on the cap and felt a shock, thinking he had been murdered. His friend explained that the cap was harmless and allowed him to understand their language. The patrolman made a comment about their Irish accents.
Dameri Tass encounters a horse and then is taken to Washington, causing a major uproar.
The United Nations demanded the alien be heard, and the Whi

IndexError: list index out of range

In [23]:
TEMPLATE_FOR_MACHINE_LEARNING = {"Developments": 
    "This briefly describes the notable and important developments due to the paper. They should be high level and "
    "not too detailed. If there are multiple developments, they should be separated by bullet points.",
    "Data": 
    " This should be a brief description of the data used in the paper. It might include the data-source names, "
    " any cleaning they did, and any new data they collected.",
    "Models":
    "This should be a brief description of the model used in the paper. It should include the type of models used, "
    "if any new architectures were introduced, and any new techniques used in the model.",
    "Training": 
    "This should be a brief description of the training process used in the paper. It should include the optimizer used, "
    "the learning rate, the batch sizes, or other things that are important to the training process, "
    " such as training from scratch or transfer learning.",
    "Results":
    "This should be a brief description of the results of the paper. It should include the evaluation metrics used, "
    "the performance of the model, and any comparisons to other models or baselines.",
    
    }


def gisting_with_template(smart_pages, query_model, 
    template=TEMPLATE_FOR_MACHINE_LEARNING, verebose=True):
    """ This takes in a function template of items that it wants to be
     sketched into separate focused 'gist' summaries about athat particular topic. 
     If a chunk has nothing to add to a a particular focus of the template, nothing is added to any subject-focused gist.

    """

    prompt_shorten_template = """
    Please shorten the following passage, focusing on the topic of {topic}, with some considerations such as the following
    
    {considerations}
    
    Just give me a shortened version if there is relevant information by saying
    
    {topic}: 

    and following it with the shortened text.
    
    DO NOT explain your reasoning or add unecessary information.

    Passage:
    {passage}

    """
    # article = example['article']
    # title = example['title']
    # word_count = count_words(article)
    # print(f"[Gisting][Article {title}], {word_count} words")
    # pages = example['pages']
    gist = {key: [] for key in template.keys()}
    for i, page in enumerate(smart_pages):
        for key, template_text in template.items():
            prompt = prompt_shorten_template.format(topic=key, considerations = template_text, passage='\n'.join(page))
            response = query_model(prompt)
            shortened_text = response.strip()
            if key in shortened_text:
                gist[key].append(shortened_text)
            # prompt = prompt_shorten_template.format(, '\n'.join(page))
            # response = query_model(prompt)
            # shortened_text = response.strip()
            # for key, template_text in template.items():
            key_marker = key + ':'

            if shortened_text.startswith(key_marker):
                gist[key].append(shortened_text[len(key_marker):].strip())
        # if verbose:
        #     print("[gist] page {}:".format(i), shortened_text, flush=True)
    # gist_word_count = {key: count_words(' '.join(value)) for key, value in gist.items()}
    # if verbose:
    #     print("Shortened article:\n", gist, flush=True)
    # output = copy.deepcopy(example)
    # output.update({'title': title, 'word_count': word_count, 'gist_word_count': gist_word_count, 'gist': gist, 'pages': pages})
    # if verbose:
    #     print(f"compression rate {round(100.0 - sum(gist_word_count.values())/word_count*100, 2)}% ({sum(gist_word_count.values())}/{word_count})")
    return gist

In [24]:
from genai.text_processes.pagination import quality_gutenberg_parser

In [66]:
%load_ext autoreload
%autoreload 2
import os
from genai.text_processes.pagination import quality_pagination


# file = 'test_text.txt'
# with open(file, 'r') as f:
#     text = f.read()
# example = {'article': text, 'title': 'Test Article'}
# pages = quality_pagination(example, query_model=llm_query)

# We will need to create something that will be able to look at arxiv .tex folders/files and parse the pages in a way that 
# allow us to do a gist on each one of the different files in the order that they are presented in the paper.

folder = '../text_processes/arXiv-2307.06435v7'

def get_text_pages_from_tex_folder(tex_folder):
    # look for the main.tex
    # Get the /sections/ in order, 
    # get the text from each section and return those as separate pages.

    main_tex = os.path.join(tex_folder, 'main.tex')
    with open(main_tex, 'r') as f:
        main_text = f.read()
    # get the sections for example:
    # \input{sections/introduction} 
    sections = re.findall(r'\\input{sections/(.*?)}', main_text)
    section_texts = []
    for section in sections:
        if section.endswith('.tex'):
            section = section[:-4]
        section_file = os.path.join(tex_folder, 'sections', section + '.tex')
        with open(section_file, 'r') as f:
            section_text = f.read()
        section_texts.append(section_text)
    return section_texts

section_texts = get_text_pages_from_tex_folder(folder)

smart_pages = []
for i,  section_text in enumerate(section_texts):
    # print(i)
    # smart_pages = section_text.split('\n')
    smart_pages += quality_gutenberg_parser(section_text)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0
1
2
3
4
5
6
7
8
9
10
11


In [64]:
section_text.split('\n')

['\\begin{IEEEbiography}{Humza Naveed}',
 'Biography text here.',
 '\\end{IEEEbiography}',
 '',
 '']

In [62]:
# smart_pages

['% Large Language Models (LLMs) have shown excellent generalization abilities that lead to the development of numerous models. These models suggest different approaches, for example, increasing model parameters, training data, tweaking architecture, training strategies, and pipelines to outperform baselines. Analyzing these aspects is important to identify changes that bring stability to the training and better generalization for the LLMs. In this paper, we summarize these fine-grained details of LLMs. Initially, we provide a background for the LLMs to discuss basic building blocks followed by a comprehensive overview of LLMs. The end of the paper discusses important findings by the LLMs and summarizes important architectural and training strategies to develop better LLMs. Because of the continuous development in LLMs, we will update this paper regularly with new sections and LLMs.  ',
 '',
 'Large Language Models (LLMs) have shown excellent generalization capabilities that have led t

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from genai.text_processes.pagination import quality_pagination


# file = 'test_text.txt'
# with open(file, 'r') as f:
#     text = f.read()
# example = {'article': text, 'title': 'Test Article'}
# pages = quality_pagination(example, query_model=llm_query)

# We will need to create something that will be able to look at arxiv .tex folders/files and parse the pages in a way that 
# allow us to do a gist on each one of the different files in the order that they are presented in the paper.

folder = '../text_processes/arXiv-2307.06435v7'

def get_text_pages_from_tex_folder(tex_folder):
    # look for the main.tex
    # Get the /sections/ in order, 
    # get the text from each section and return those as separate pages.

    main_tex = os.path.join(tex_folder, 'main.tex')
    with open(main_tex, 'r') as f:
        main_text = f.read()
    # get the sections for example:
    # \input{sections/introduction} 
    sections = re.findall(r'\\input{sections/(.*?)}', main_text)
    section_texts = []
    for section in sections:
        if section.endswith('.tex'):
            section = section[:-4]
        section_file = os.path.join(tex_folder, 'sections', section + '.tex')
        with open(section_file, 'r') as f:
            section_text = f.read()
        section_texts.append(section_text)
    return section_texts

section_texts = get_text_pages_from_tex_folder(folder)

smart_pages = []
for i,  section_text in enumerate(section_texts):
    print(i)
    smart_pages += section_text.split('\n')
smart_pages = quality_gutenberg_parser(smart_pages)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0
1
2
3
4
5
6
7
8
9
10
11


AttributeError: 'list' object has no attribute 'split'

In [41]:
quality_pagination
gists = gisting_with_template(smart_pages, llm_query)

NameError: name 'paragraphs' is not defined

In [22]:
result_gists = gisting_with_template(pages, query_model=llm_query)

TypeError: gisting_with_template() missing 1 required positional argument: 'template'